## Analyse the competing phases

We load the search structures found by atomate and then compare against the competing phases relaxed using AiiDA.

In [1]:
%aiida

Loaded AiiDA DB environment - profile name: scanlon-test.

In [2]:
from toolchest import hull

import pandas as pd

## Load the entries from the previous calculations

In [5]:
li_fe_o_group = Group.objects.get(label='li-fe-o-s/mp-structures')
c_f_group = Group.objects.get(label='oxalates-f/competing/li-fe-c-o-f-phases/with-c-f')

def get_valid_pd(groups):
    entries = hull.get_relax_records(groups, 520, functional='pbesol')

    valid_entries = entries[
        (entries.functional == 'pbesol') & 
        (~entries.formula.str.contains('S') )
        ]
    valid_rows = []
    for _, row in valid_entries.iterrows():
        if 'Fe' in row.formula:
            if row.umap.get('Fe') == 4.0:
                valid_rows.append(row)
        else:
            valid_rows.append(row)
    valid_entries = pd.DataFrame(valid_rows)
    return valid_entries

In [6]:
competing_df = get_valid_pd([li_fe_o_group, c_f_group])

Entries: 209


  0%|          | 0/209 [00:00<?, ?it/s]

In [7]:
competing_entries = hull.get_entry(competing_df)

## Experimental structure of $\ce{Li2Fe(C2O4)2}$

In [8]:
from importlib import reload
reload(hull)

<module 'toolchest.hull' from '/home/bonan/appdir/python-tools/toolchest/hull.py'>

In [10]:
relax=  load_node('900cca7e-baba-4590-ad63-71971e1fc681')
exp_df = hull.get_relax_record_single(relax)

exp_df['energy_per_atom'] = exp_df['energy'] / exp_df.pmg_struct.apply(lambda x: x.composition.num_atoms)

## Load search structures

In [22]:
from monty.serialization import dumpfn, loadfn

In [23]:
search_entries = loadfn("search_entries.json")

## Construct the phase diagram

In [25]:
from pymatgen.analysis.phase_diagram import PhaseDiagram, CompoundPhaseDiagram, PDPlotter
from pymatgen.analysis.reaction_calculator import ComputedReaction

In [26]:


for entry in competing_entries:
    entry.correction = 0

for entry in search_entries:
    entry.correction = 0

        
pd_entries = search_entries + competing_entries

phased = PhaseDiagram(pd_entries)

In [27]:
for entry in pd_entries:
    if entry.composition.reduced_formula == 'LiF':
        lif = entry
    if entry.composition.reduced_formula == 'FeF2':
        fef2 = entry
    if entry.composition.reduced_formula == 'LiCO2':
        lico2 = entry
    if entry.composition.reduced_formula == 'FeCO3':
        feco3 = entry
    if entry.composition.reduced_formula == 'Fe(CO2)2':
        feox = entry

In [28]:
search_entries.sort(key=lambda x: phased.get_e_above_hull(x))
comps = {}
for entry in search_entries + [lico2]:
    decomp, e_above_hull = phased.get_decomp_and_e_above_hull(entry)
    decomp_forms = [x.composition.reduced_formula for x in decomp.keys()]
    if entry.composition.reduced_formula not in comps:
        print(f'{entry.composition.reduced_formula:<15} {str(decomp_forms):<50} Above hull: {e_above_hull:>8.3f} eV')
        comps[entry.composition.reduced_formula] = entry
    entry.data['e_above_hull'] = e_above_hull # Store the energy above the hull
#print(comps)

Li2Fe(CO2)4     ['LiFe(CO3)2', 'Li2CO3', 'CO2', 'C']               Above hull:    0.051 eV
Li2FeC2(O2F)2   ['FeCO3', 'CO2', 'LiF', 'C']                       Above hull:    0.055 eV
Fe(CO2)2        ['FeCO3', 'CO2', 'C']                              Above hull:    0.067 eV
Li3FeC4O8F      ['LiFe(CO3)2', 'Li2CO3', 'CO2', 'LiF', 'C']        Above hull:    0.081 eV
LiFeC2O4F       ['FeCO3', 'CO2', 'LiF', 'C']                       Above hull:    0.083 eV
LiCO2           ['Li2CO3', 'CO2', 'C']                             Above hull:    0.047 eV
